# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [2]:
# your code here
import pandas as pd
import numpy as np

tic= pd.read_csv('tic-tac-toe.csv')

In [3]:
# Show the first few rows
print('First 5 rows:')
print(tic.head())

# Check for missing values
print('\nMissing Values:')
print(tic.isnull().sum())

# Check unique values in each column
print('\nUnique Values in each column:')
for col in tic.columns:
    print(f"{col} unique values:", tic[col].unique())

print('\nDataste Info')
print(tic.info())

First 5 rows:
  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True

Missing Values:
TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

Unique Values in each column:
TL unique values: ['x' 'o' 'b']
TM unique values: ['x' 'o' 'b']
TR unique values: ['x' 'o' 'b']
ML unique values: ['x' 'o' 'b']
MM unique values: ['o' 'b' 'x']
MR unique values: ['o' 'b' 'x']
BL unique values: ['x' 'o' 'b']
BM unique values: ['o' 'x' 'b']
BR unique values: ['o' 'x' 'b']
class unique values: [ True False]

Dataste Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      

In [4]:
# No missing values in any column.
# All board columns (TL, TM, etc.) have only 'x', 'o', or 'b' as values.
# Target column (class) is boolean (True/False).
# Data types are as expected.
# Conclusion:
# The dataset is clean and reliable for modeling. 

In [5]:
# Convert categorical values to numeric
# x=1, o=-1, b=0
mapping = {'x': 1, 'o': -1, 'b': 0}
for col in tic.columns[:-1]:
    tic[col] = tic[col].map(mapping)

# Convert class column to int (True=1, False=0)
tic['class'] = tic['class'].astype(int)

# Separate inputs and output
X = tic.drop('class', axis=1).values
y = tic['class'].values

# # Normalize input data: map -1, 0, 1 to 0, 0.5, 1
# Example: X = (X + 1) / 2  # maps -1,0,1 to 0,0.5,1
X = (X + 1) / 2

# Check shapes
print(X.shape, y.shape)

(958, 9) (958,)


In [6]:
# Examine the processed features and target
print("First 5 rows of X (features):")
print(X[:5])

print("\nFirst 5 values of y (target):")
print(y[:5])

print("\nFeature value range (min, max):")
print(X.min(), X.max())

First 5 rows of X (features):
[[1.  1.  1.  1.  0.  0.  1.  0.  0. ]
 [1.  1.  1.  1.  0.  0.  0.  1.  0. ]
 [1.  1.  1.  1.  0.  0.  0.  0.  1. ]
 [1.  1.  1.  1.  0.  0.  0.  0.5 0.5]
 [1.  1.  1.  1.  0.  0.  0.5 0.  0.5]]

First 5 values of y (target):
[1 1 1 1 1]

Feature value range (min, max):
0.0 1.0


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [7]:
# your code here
# Step 2: Build Neural Network

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input


# 1. Split the training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Create a Sequential model
model = Sequential()
model.add(Input(shape=(9,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))  # 2 classes: win or not win

# 3. Compile the model
model.compile(optimizer='adam',
			  loss='sparse_categorical_crossentropy',
			  metrics=['accuracy'])

# 4. Fit the training data
history = model.fit(X_train, y_train, epochs=30, batch_size=16, validation_split=0.1, verbose=1)

# 5. Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# 6. Save the model
model.save('tic-tac-toe.keras')

Epoch 1/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6589 - loss: 0.6091 - val_accuracy: 0.7273 - val_loss: 0.5400
Epoch 2/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7039 - loss: 0.5692 - val_accuracy: 0.7662 - val_loss: 0.5159
Epoch 3/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7257 - loss: 0.5429 - val_accuracy: 0.7662 - val_loss: 0.5039
Epoch 4/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7257 - loss: 0.5244 - val_accuracy: 0.7792 - val_loss: 0.4902
Epoch 5/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7344 - loss: 0.5117 - val_accuracy: 0.7922 - val_loss: 0.4787
Epoch 6/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7475 - loss: 0.4965 - val_accuracy: 0.8052 - val_loss: 0.4700
Epoch 7/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7620 - loss: 0.4819 - val_accuracy: 0.8052 - val_loss: 0.4610
Epoch 8/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7852 - loss: 0.4656 - val_accuracy: 0.8052 - val_loss:

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [8]:
# your code here
from tensorflow.keras.models import load_model


# Load the saved model
model = load_model('tic-tac-toe.keras')

# Pick 5 random samples from the test set
idx = np.random.choice(len(X_test), 5, replace=False)
X_sample = X_test[idx]
y_true = y_test[idx]

# Make predictions
y_pred_probs = model.predict(X_sample)
y_pred = np.argmax(y_pred_probs, axis=1)

print("True labels:     ", y_true)
print("Predicted labels:", y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
True labels:      [1 1 1 0 1]
Predicted labels: [1 1 1 0 1]


In [9]:
# Predict on all test data
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Compare predictions to true labels
print("True labels:     ", y_test)
print("Predicted labels:", y_pred)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
True labels:      [0 1 1 0 0 1 1 1 1 0 1 1 1 0 1 1 1 1 0 0 0 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 1
 1 1 0 1 1 0 0 1 0 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 0 1 1 0 0 0 1
 0 1 1 0 1 0 1 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1 0 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 0
 1 1 1 0 0 0 1 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 0 1 1 0 0 1
 1 0 1 0 1 1 0]
Predicted labels: [0 1 1 0 0 1 1 1 1 0 1 1 1 0 1 1 1 1 0 0 0 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 1
 1 1 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 0 1 1 0 0 0 1
 0 1 1 0 1 0 1 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 1 0 1 1 0 1 0 0 1 0 1 0 1 1 0 1 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 1 0 1 1 0
 1 1 1 0 0 0 1 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 0 1 1 0 0 1
 1 0 1 0 1 0 0]


In [10]:
#Summary report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97        67
           1       0.99      0.98      0.98       125

    accuracy                           0.98       192
   macro avg       0.97      0.98      0.98       192
weighted avg       0.98      0.98      0.98       192



## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [11]:
# your code here
#No the model didn't achieve the desired results.
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

# 1. Split the training and test data (already done)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Create a deeper Sequential model
model = Sequential()
model.add(Input(shape=(9,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))


# 3. Compile with a lower learning rate
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 4. Fit the training data with more epochs
history = model.fit(X_train, y_train, epochs=60, batch_size=16, validation_split=0.1, verbose=1)

# 5. Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# 6. Save the improved model
model.save('tic-tac-toe2.keras')

Epoch 1/60
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6459 - loss: 0.6433 - val_accuracy: 0.7143 - val_loss: 0.5677
Epoch 2/60
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6589 - loss: 0.5920 - val_accuracy: 0.8052 - val_loss: 0.5360
Epoch 3/60
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7083 - loss: 0.5657 - val_accuracy: 0.7922 - val_loss: 0.5262
Epoch 4/60
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7388 - loss: 0.5419 - val_accuracy: 0.7662 - val_loss: 0.5127
Epoch 5/60
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7300 - loss: 0.5417 - val_accuracy: 0.8052 - val_loss: 0.4834
Epoch 6/60
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7388 - loss: 0.5135 - val_accuracy: 0.8312 - val_loss: 0.4637
Epoch 7/60
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7562 - loss: 0.5048 - val_accuracy: 0.7922 - val_loss: 0.4702
Epoch 8/60
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7939 - loss: 0.4597 - val_accuracy: 0.8182 - val_loss:

In [12]:
#Summary report
from sklearn.metrics import classification_report

# Predict on all test data
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

print(classification_report(y_test, y_pred))

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
              precision    recall  f1-score   support

           0       1.00      0.99      0.99        67
           1       0.99      1.00      1.00       125

    accuracy                           0.99       192
   macro avg       1.00      0.99      0.99       192
weighted avg       0.99      0.99      0.99       192



**Which approach(es) did you find helpful to improve your model performance?**

In [13]:
# your answer here
# Increasing the number of layers and neurons, using dropout for regularization, 
# increasing the number of epochs, and lowering the learning rate all helped improve model performance.
# The final model achieved high accuracy and low loss.